### Imports and configs

In [17]:
from opinf_for_hw.data_proc import *
from opinf_for_hw.postproc import *
from opinf_for_hw.utils.helpers import loader
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import xarray as xr
import h5py

cluster = True
animations = False

if cluster:
    print(f"\033[1m Using cluster settings \033[0m")
    from config.cluster import *
else:
    print(f"\033[1m Using local settings \033[0m")
    from config.local import *
r=78
print(f"\033[1m Using r={r} \033[0m")

 Using cluster settings 
 Using r=78 


### Helpers

In [2]:
def solve_opinf_difference_model(s0, n_steps, f):
    s = np.zeros((np.size(s0), n_steps))
    is_nan = False

    s[:, 0] = s0
    for i in range(n_steps - 1):
        s[:, i + 1] = f(s[:, i])

        if np.any(np.isnan(s[:, i + 1])):
            print("NaN encountered at iteration " + str(i + 1))
            is_nan = True
            break

    return is_nan, s

### Algorithm

#### Load the data

In [3]:
print("\033[1m Prepare the data for learning (MULTI-IC version) \033[0m")

# Load combined training data from multiple initial conditions
Xhat_train_file = output_path + "X_hat_train_multi_IC.npy"
Xhatmax = np.load(Xhat_train_file)
Xhat = Xhatmax[:, :r]

print(f"Training data shape: {Xhat.shape}")

# Prepare state evolution data
X_state = Xhat[:-1, :]
Y_state = Xhat[1:, :]

s = int(r * (r + 1) / 2)
d_state = r + s
d_out = r + s + 1

X_state2 = get_x_sq(X_state)
D_state = np.concatenate((X_state, X_state2), axis=1)
D_state_2 = D_state.T @ D_state
print("\033[1m State learning data prepared \033[0m")

 Prepare the data for learning (MULTI-IC version) 
Training data shape: (16001, 78)
 State learning data prepared 


In [4]:
print("\033[1m Prepare the output learning data \033[0m")

X_out = Xhatmax[:, :r]
K = X_out.shape[0]
E = np.ones((K, 1))

mean_Xhat = np.mean(X_out, axis=0)
Xhat_out = X_out - mean_Xhat[np.newaxis, :]

local_min = np.min(X_out)
local_max = np.max(X_out)
local_scaling = np.maximum(np.abs(local_min), np.abs(local_max))
scaling_Xhat = local_scaling

Xhat_out /= scaling_Xhat
Xhat_out2 = get_x_sq(Xhat_out)

D_out = np.concatenate((Xhat_out, Xhat_out2, E), axis=1)
D_out_2 = D_out.T @ D_out

print(f"D_out shape: {D_out.shape}")
print(f"D_out_2 condition number: {np.linalg.cond(D_out_2):.2e}")
print("\033[1m Done \033[0m")

 Prepare the output learning data 
D_out shape: (16001, 3160)
D_out_2 condition number: 6.03e+08
 Done 


In [5]:
print("\033[1m Load derived quantities from all training trajectories \033[0m")

ENGINE = "h5netcdf"

Gamma_n_list = []
Gamma_c_list = []

for file_path in training_files:
    fh = loader(file_path, ENGINE=ENGINE)
    Gamma_n_list.append(fh["gamma_n"].data)
    Gamma_c_list.append(fh["gamma_c"].data)

# Concatenate all trajectories
Gamma_n = np.concatenate(Gamma_n_list)
Gamma_c = np.concatenate(Gamma_c_list)

mean_Gamma_n_ref = np.mean(Gamma_n)
std_Gamma_n_ref = np.std(Gamma_n, ddof=1)

mean_Gamma_c_ref = np.mean(Gamma_c)
std_Gamma_c_ref = np.std(Gamma_c, ddof=1)

Y_Gamma = np.vstack((Gamma_n, Gamma_c))

print(f"Gamma_n shape: {Gamma_n.shape}")
print(f"Gamma_c shape: {Gamma_c.shape}")
print(f"Y_Gamma shape: {Y_Gamma.shape}")
print(f"X_out shape (for output learning): {X_out.shape}")
print(f"Shape compatibility check: Y_Gamma cols ({Y_Gamma.shape[1]}) vs X_out rows ({X_out.shape[0]})")

if Y_Gamma.shape[1] != X_out.shape[0]:
    raise ValueError(f"Shape mismatch: Y_Gamma has {Y_Gamma.shape[1]} columns but X_out has {X_out.shape[0]} rows")

print(f"Mean Gamma_n: {mean_Gamma_n_ref:.4f}, Std: {std_Gamma_n_ref:.4f}")
print(f"Mean Gamma_c: {mean_Gamma_c_ref:.4f}, Std: {std_Gamma_c_ref:.4f}")

# Check for NaNs in gamma data
if np.any(np.isnan(Gamma_n)):
    print("\033[91m WARNING: NaNs detected in Gamma_n training data! \033[0m")
if np.any(np.isnan(Gamma_c)):
    print("\033[91m WARNING: NaNs detected in Gamma_c training data! \033[0m")
if np.any(np.isinf(Gamma_n)):
    print("\033[91m WARNING: Infs detected in Gamma_n training data! \033[0m")
if np.any(np.isinf(Gamma_c)):
    print("\033[91m WARNING: Infs detected in Gamma_c training data! \033[0m")

print("\033[1m Done \033[0m")

 Load derived quantities from all training trajectories 
 ERROR: Could not open file /work2/10407/anthony50102/frontera/data/hw2d_sim/t600_d256x256_raw/hw2d_sim_step0.025_end1_pts512_c11_k015_N3_nu5e-8_20250315142044_11702_0.h5: variable '/density' has no dimension scale associated with axis 0. 
Use phony_dims='sort' for sorted naming or phony_dims='access' for per access naming. 
  Retrying with phony_dims='sort'...
Gamma_n shape: (16001,)
Gamma_c shape: (16001,)
Y_Gamma shape: (2, 16001)
X_out shape (for output learning): (16001, 78)
Shape compatibility check: Y_Gamma cols (16001) vs X_out rows (16001)
Mean Gamma_n: 0.5901, Std: 0.0416
Mean Gamma_c: 0.5843, Std: 0.0354
 Done 


In [6]:
print("\033[1m Load test initial condition \033[0m")

IC_data = np.load(output_path + "initial_conditions_multi_IC.npz")
test_IC_reduced = IC_data["test_ICs_reduced"][:r]

print(f"Test IC shape: {test_IC_reduced.shape}")

 Load test initial condition 
Test IC shape: (1, 16001)


#### Run the parameter sweep

In [7]:
print("\033[1m BEGIN PARAMETER SWEEP - TRACKING BEST L2 ERROR \033[0m")

best_l2_error = np.inf
best_model = None
best_alphas = None

for alpha_state_lin in ridge_alf_lin_all:
    for alpha_state_quad in ridge_alf_quad_all:
        print("alpha_lin = %.2E" % alpha_state_lin)
        print("alpha_quad = %.2E" % alpha_state_quad)

        regg = np.zeros(d_state)
        regg[:r] = alpha_state_lin
        regg[r : r + s] = alpha_state_quad
        regularizer = np.diag(regg)
        D_state_reg = D_state_2 + regularizer

        O = np.linalg.solve(D_state_reg, np.dot(D_state.T, Y_state)).T

        A = O[:, :r]
        F = O[:, r : r + s]

        # Compute training predictions
        Y_state_pred = A @ X_state.T + F @ X_state2.T
        Y_state_pred = Y_state_pred.T

        # Check for NaNs
        if np.any(np.isnan(Y_state_pred)) or np.any(np.isinf(Y_state_pred)):
            print("  Skipping due to NaN/Inf in training predictions")
            continue

        # Compute L2 error on training data
        l2_error = np.linalg.norm(Y_state - Y_state_pred, 'fro')
        relative_l2_error = l2_error / np.linalg.norm(Y_state, 'fro')

        print(f"  L2 error: {l2_error:.6e} | Relative L2: {relative_l2_error:.6e}")

        # # Learn output operators for this state model
        # for n, alpha_out_lin in enumerate(gamma_reg_lin):
        #     for m, alpha_out_quad in enumerate(gamma_reg_quad):
        #         regg = np.zeros(d_out)
        #         regg[:r] = alpha_out_lin
        #         regg[r : r + s] = alpha_out_quad
        #         regg[r + s :] = alpha_out_lin
        #         regularizer = np.diag(regg)
        #         D_out_reg = D_out_2 + regularizer

        #         # Check condition number
        #         cond_num = np.linalg.cond(D_out_reg)
        #         if cond_num > 1e15:
        #             continue

        #         O_out = np.linalg.solve(D_out_reg, np.dot(D_out.T, Y_Gamma.T)).T

        #         # Check for NaNs in output operators
        #         if np.any(np.isnan(O_out)) or np.any(np.isinf(O_out)):
        #             continue

        #         C = O_out[:, :r]
        #         G = O_out[:, r : r + s]
        #         c = O_out[:, r + s]

        # If this is the best model so far, save it
        if l2_error < best_l2_error:
            best_l2_error = l2_error
            best_model = {
                'A': A.copy(),
                'F': F.copy(),
                # 'C': C.copy(),
                # 'G': G.copy(),
                # 'c': c.copy(),
                'relative_l2_error': relative_l2_error,
                'absolute_l2_error': l2_error
            }
            best_alphas = {
                'alpha_state_lin': alpha_state_lin,
                'alpha_state_quad': alpha_state_quad,
                # 'alpha_out_lin': alpha_out_lin,
                # 'alpha_out_quad': alpha_out_quad
            }
            print(f"  ✓ New best model! L2 error: {l2_error:.6e}")

print("\033[1m Done \033[0m")

 BEGIN PARAMETER SWEEP - TRACKING BEST L2 ERROR 
alpha_lin = 1.00E+04
alpha_quad = 1.00E+05
  L2 error: 1.686048e+02 | Relative L2: 2.075567e-03
  ✓ New best model! L2 error: 1.686048e+02
alpha_lin = 1.00E+04
alpha_quad = 9.00E+04
  L2 error: 1.686425e+02 | Relative L2: 2.076032e-03
alpha_lin = 1.00E+04
alpha_quad = 8.00E+04
  L2 error: 1.686847e+02 | Relative L2: 2.076551e-03
alpha_lin = 1.00E+04
alpha_quad = 7.00E+04
  L2 error: 1.687316e+02 | Relative L2: 2.077128e-03
alpha_lin = 1.00E+04
alpha_quad = 6.00E+04
  L2 error: 1.687834e+02 | Relative L2: 2.077766e-03
alpha_lin = 1.00E+04
alpha_quad = 5.00E+04
  L2 error: 1.688405e+02 | Relative L2: 2.078469e-03
alpha_lin = 1.00E+04
alpha_quad = 4.00E+04
  L2 error: 1.689032e+02 | Relative L2: 2.079241e-03
alpha_lin = 1.00E+04
alpha_quad = 3.00E+04
  L2 error: 1.689718e+02 | Relative L2: 2.080085e-03
alpha_lin = 1.00E+04
alpha_quad = 2.00E+04
  L2 error: 1.690467e+02 | Relative L2: 2.081007e-03
alpha_lin = 1.00E+04
alpha_quad = 1.00E+04
 

#### Save the best model

In [8]:
print(f"\n\033[1m BEST MODEL RESULTS \033[0m")
if best_model is not None:
    print(f"Best L2 error: {best_model['absolute_l2_error']:.6e}")
    print(f"Best relative L2 error: {best_model['relative_l2_error']:.6e}")
    print(f"Best regularization parameters:")
    print(f"  alpha_state_lin:  {best_alphas['alpha_state_lin']:.2e}")
    print(f"  alpha_state_quad: {best_alphas['alpha_state_quad']:.2e}")
    
    # Save the best model
    np.savez(
        output_path + "best_model_l2_multi_IC_r" + str(r) + ".npz",
        A=best_model['A'],
        F=best_model['F'],
        alpha_state_lin=best_alphas['alpha_state_lin'],
        alpha_state_quad=best_alphas['alpha_state_quad'],
        l2_error=best_model['absolute_l2_error'],
        relative_l2_error=best_model['relative_l2_error']
    )
    print(f"\033[1m Best model saved to {output_path}/best_model_l2_multi_IC_r" + str(r) + ".npz \033[0m")
    
    # Save predictions from the best model
    Y_state_pred = best_model['A'] @ X_state.T + best_model['F'] @ X_state2.T
    Y_state_pred = Y_state_pred.T
    np.savez(
            output_path + "Y_state_best_model_l2_multi_IC_r" + str(r) + ".npz",
            Y_state_pred
    )
    
    ###########################
    print("\033[1m Reconstruct training data in original space \033[0m")
    # Load POD basis
    POD_data = np.load(output_path + "POD_multi_IC.npz")
    Vr = POD_data["Vr"][:, :r]  # Shape: (2*64*64, r)
    
    # Get POD predictions from best model
    A_best = best_model['A']
    F_best = best_model['F']
    
    # Reconstruct state trajectory in POD space (one-step predictions)
    Y_state_pred = A_best @ X_state.T + F_best @ X_state2.T
    X_recon_pod = Y_state_pred.T  # Shape: (n_timesteps-1, r)
    
    # Add the initial condition to get full trajectory
    X_full_pod = np.vstack([X_state[0, :], X_recon_pod])  # Shape: (n_timesteps, r)
    
    # Project back to full space
    X_full_space = X_full_pod @ Vr.T  # Shape: (n_timesteps, 2*64*64)
    print(f"Reconstructed data shape: {X_full_space.shape}")
    print(f"  (represents {X_full_space.shape[0]} timesteps × {X_full_space.shape[1]//2} spatial points × 2 fields)")
    
    ###########################
    print("\033[1m Create ground truth POD reconstruction for comparison \033[0m")
    # Load original training data (you'll need to adjust this path)
    # Assuming you have the original data in a variable or file
    # For example, if your original data is in X_train_original with shape (n_timesteps, 2*64*64)
    # X_train_original = ... (load your original training data here)
    
    # Project original data to POD space and back for fair comparison
    X_train_original = np.array
    with h5py.File(training_files[0], 'r') as f:
        density = np.expand_dims(f['density'][:], axis=1)
        pot = np.expand_dims(f['phi'][:], axis=1)
        X_train_original = np.concatenate((density, pot), axis=1).reshape(density.shape[0], -1)
        
    X_truth_pod = X_train_original @ Vr  # Project down: (n_timesteps, r)
    X_truth_recon = X_truth_pod @ Vr.T    # Project back up: (n_timesteps, 2*64*64)
    
    print(f"Ground truth POD reconstruction shape: {X_truth_recon.shape}")
    
    # Save reconstruction and ground truth
    np.savez(
        output_path + "training_reconstruction_multi_IC_r" + str(r) + ".npz",
        X_recon_full=X_full_space,      # Model prediction reconstruction
        X_truth_full=X_truth_recon,     # Ground truth POD reconstruction
        X_recon_pod=X_full_pod,         # POD coefficient trajectory (predictions)
        X_truth_pod=X_truth_pod,        # POD coefficient trajectory (ground truth)
        n_timesteps=X_full_space.shape[0],
        n_spatial=X_full_space.shape[1]//2,
        n_fields=2
    )
    print("\033[1m Training reconstruction and ground truth saved! \033[0m")
else:
    print("\033[1m WARNING: No valid models found! \033[0m")


 BEST MODEL RESULTS 
Best L2 error: 1.393984e+02
Best relative L2 error: 1.716030e-03
Best regularization parameters:
  alpha_state_lin:  1.00e-04
  alpha_state_quad: 1.00e+05
 Best model saved to /work2/10407/anthony50102/frontera/data/sciml_roms_hasegawa_wakatani//best_model_l2_multi_IC_r78.npz 
 Reconstruct training data in original space 
Reconstructed data shape: (16001, 131072)
  (represents 16001 timesteps × 65536 spatial points × 2 fields)
 Create ground truth POD reconstruction for comparison 
Ground truth POD reconstruction shape: (16001, 131072)
 Training reconstruction and ground truth saved! 


### Evaluate the model

In [9]:
### Load the prediction and ground truth
pred_file = np.load(output_path + "training_reconstruction_multi_IC_r" + str(r) + ".npz")
print(pred_file.keys())
X_recon = pred_file['X_recon_full']
X_truth = pred_file['X_truth_full']
print(f"Reconstruction shape: {X_recon.shape}")
print(f"Ground truth shape: {X_truth.shape}")

KeysView(NpzFile '/work2/10407/anthony50102/frontera/data/sciml_roms_hasegawa_wakatani/training_reconstruction_multi_IC_r78.npz' with keys: X_recon_full, X_truth_full, X_recon_pod, X_truth_pod, n_timesteps...)
Reconstruction shape: (16001, 131072)
Ground truth shape: (16001, 131072)


In [18]:
if animations:
    from IPython.display import HTML
    plt.rcParams['animation.embed_limit'] = 250
    
    # Create 2x2 subplot grid
    fig, axes = plt.subplots(2, 2, figsize=(12, 12))
    ax = axes.flatten()
    
    # Downsample
    X_recon_subbed = X_recon[::100].reshape(-1, 2, 256, 256)
    X_truth_subbed = X_truth[::100].reshape(-1, 2, 256, 256)
    
    # Initialize images
    # Top row: Predictions
    im0 = ax[0].imshow(X_recon_subbed[0, 0], animated=True, cmap='viridis')
    ax[0].set_title('Prediction - Field 1')
    ax[0].axis('off')
    
    im1 = ax[1].imshow(X_recon_subbed[0, 1], animated=True, cmap='viridis')
    ax[1].set_title('Prediction - Field 2')
    ax[1].axis('off')
    
    # Bottom row: Ground Truth
    im2 = ax[2].imshow(X_truth_subbed[0, 0], animated=True, cmap='viridis')
    ax[2].set_title('Ground Truth - Field 1')
    ax[2].axis('off')
    
    im3 = ax[3].imshow(X_truth_subbed[0, 1], animated=True, cmap='viridis')
    ax[3].set_title('Ground Truth - Field 2')
    ax[3].axis('off')
    
    plt.tight_layout()
    
    def animate(frame):
        im0.set_data(X_recon_subbed[frame, 0])
        im1.set_data(X_recon_subbed[frame, 1])
        im2.set_data(X_truth_subbed[frame, 0])
        im3.set_data(X_truth_subbed[frame, 1])
        return (im0, im1, im2, im3)
    
    ani = animation.FuncAnimation(
        fig, animate,
        frames=X_recon_subbed.shape[0],
        interval=20,
        blit=True
    )
    
    ani.save(output_path + "training_reconstruction_multi_IC_r" + str(r) + ".gif", writer="pillow")
    print(f"Saved the animation to: {output_path}training_reconstruction_multi_IC_r{r}.gif")
    
    HTML(ani.to_jshtml())

else:
    print("Not showing animations")

Not showing animations
